In [130]:
%matplotlib inline

In [379]:
import pandas as pd

def qanswers(q_num):
    return data[data["question id"] == q_num]\
     .loc[:, ["user_key", "question text", "answer_cleaned"]]\
     .groupby(["question text", "answer_cleaned"])\
     .agg("count")\
     .groupby(level=0)\
     .apply(lambda x : x / x.sum()) * 100



data = pd.read_csv("answers_clean.csv")

gender_questions = data[data["question text"] == "What is your gender?"]
gender_questions = gender_questions.set_index("user_key")
genders = gender_questions.loc[:,"answer"]

age_questions = data[data["question text"] == "What is your year of birth?"]
age_questions = age_questions.set_index("user_key")
ages = age_questions.loc[:, "answer"]

data = data.join(genders, on="user_key", rsuffix="_gender")
data = data.join(ages, on="user_key", rsuffix="_yob")

data = data.rename({"answer_gender": "gender", "answer_yob": "birth_year"}, axis=1)
data["birth_year"] = data["birth_year"].astype(float)
data["age"] = 2019 - data["birth_year"]



How many people have ever answered a question on Khetha?

In [132]:
len(set(data.user_key))

670

Gender and age breakdown of all people who answered those questions

In [153]:
gender_questions.groupby("answer").agg("count")

,submission id,submission complete,timestamp,task slug,question id,question text,answer_cleaned,Province
answer,,,,,,,,
Female,255,255,255,255,255,255,255,215
I would rather not say,3,3,3,3,3,3,3,2
Male,281,281,281,281,281,281,281,235
Other,1,1,1,1,1,1,1,0


Spatial distribution of all people for whom we have that info

In [151]:
data.loc[:, ["Province", "user_key"]].drop_duplicates().groupby("Province").agg("count")


,user_key
Province,
Eastern Cape,37
Free State,24
Gauteng,233
KwaZulu-Natal,56
Limpopo,28
Mpumalanga,20
North West,20
Northern Cape,8
Western Cape,35


Total people who may have joined the app for the first few questions (e.g. Big Debate) but never answered the demographic and location questions. - People who haven't answered the gender question

In [169]:
data.loc[:, ["question id", "question text"]]
DEMOGRAPHICS_SLUG = "tell-us-about-yourself"
demographics_questions = data[data["task slug"] == DEMOGRAPHICS_SLUG]
demographics_users = len(set(demographics_questions["user_key"]))
all_users = len(set(data["user_key"]))

print(f"All users: {all_users}, no demographics: {all_users - demographics_users}")

All users: 670, no demographics: 122


How many people completed all questions and modules

In [188]:
data.loc[:, ["user_key", "task slug"]]\
 .drop_duplicates()\
 .groupby("user_key")\
 .agg("count")\
 .reset_index()\
 .sort_values("task slug")\
 .groupby("task slug")\
 .agg("count")

,user_key
task slug,
1,82
2,22
3,73
4,29
5,31
6,39
7,28
8,31
9,11


Total number of respondents for each module (so we can see drop-off by module)

In [199]:
data.loc[:, ["user_key", "task slug"]]\
 .drop_duplicates()\
 .reset_index()\
 .groupby("task slug")\
 .agg("count")\
 .rename(columns={"index" : "count"})\
 .loc[:, "count"]

task slug
BD-1                                  162
BD-2                                  156
BD-3                                  154
BD-4                                  159
BD-5                                  153
BD-6                                  154
BD-7                                  153
BD-8                                  157
Constitution                          295
Voting-2                              297
barrier-to-voting                     286
contact-details                       502
election-importance                   482
fact-or-emotion                       334
have-your-say                         172
help-someone-vote                     225
how-do-you-feel-about-south-africa    175
inter-generational-conversation       255
meet-your-local-party                 188
parties-competing                     428
policy-positions-manisfestos          314
post-election-follow-up               199
priority-policy-issues                248
reasons-for-supporting-p

For each module, how many people started the module but didn’t finish it (if that can be seen)

In [238]:
task_questions = data\
 .loc[:, ["task slug", "question text"]]\
 .drop_duplicates()\
 .groupby("task slug")\
 .agg("count")

user_answers = data.loc[:, ["user_key", "task slug", "question text"]]\
 .groupby(["user_key", "task slug"])\
 .agg("count")

task_responses = user_answers.join(task_questions, on="task slug", rsuffix="_right")
task_responses["complete"] = task_responses["question text"] == task_responses["question text_right"]
task_responses.reset_index()\
 .loc[:, ["user_key", "task slug", "complete"]]\
 .groupby(["task slug", "complete"])\
 .agg("count")

,,user_key
task slug,complete,
BD-1,True,162
BD-2,True,156
BD-3,True,154
BD-4,True,159
BD-5,True,153
BD-6,True,154
BD-7,True,153
BD-8,True,157
Constitution,True,295


How many people clicked through from FB posts and then a) answered some questions b) did not convert into any participation beyond the click

* What is your MAIN reason for choosing a party to vote for? (all responses before 8 May)
* What was your main reason for choosing the party you voted for? (all responses after elections)

In [266]:
data.loc[:, ["question id", "question text"]].set_index("question id")#.reset_index()#.drop_duplicates()
Q_MAIN_REASON_BEFORE = 26
Q_MAIN_REASON_AFTER = 151
reasons_before = data[data["question id"] == Q_MAIN_REASON_BEFORE].loc[:, ["user_key", "answer"]].set_index("user_key")
reasons_after = data[data["question id"] == Q_MAIN_REASON_AFTER].loc[:, ["user_key", "answer"]].set_index("user_key")
answers_join = reasons_before.join(reasons_after, rsuffix="_after")
answers_join.reset_index().groupby(["answer", "answer_after"]).agg("count").rename(columns={"user_key": "# users"})

# users
answer                                             answer_after                                               
I agree with their proposed policies on issues ... I did not vote                                            7
                                                   I had to vote (no particular reason)                      1
                                                   I wanted the policies I cared about to be actioned       12
                                                   I wanted to support the party I felt has done t...        6
                                                   I wanted to support the party that has helped m...        3
                                                   We needed change in leadership                            9
I am part of an organisation that supports this... I wanted to support the party that has helped m...        1
I trust the party representatives                  I had to vote (no particular reason)                      1
                                                   I wanted the policies I cared about to be actioned        2
                                                   I wanted to support the party I felt has done t...        4
                                                   I wanted to support the party that has helped m...        3
                                                   We needed change in leadership                            2
My family and/or friends support the party         I wanted to support the party I felt has done t...        3
                                                   I wanted to support the party that has helped m...        2
                                                   We needed change in leadership                            1
None of the above                                  I did not vote                                            8
                                                   I had to vote (no particular reason)                      1
                                                   I wanted the policies I cared about to be actioned        1
                                                   I wanted to support the party I felt has done t...        2
                                                   I wanted to support the party that has helped m...        1
                                                   We needed change in leadership                            1
The other parties are worse                        I did not vote                                            1
                                                   I wanted the policies I cared about to be actioned        1
                                                   I wanted to support the party I felt has done t...        2
                                                   I wanted to support the party that has helped m...        1
                                                   We needed change in leadership                            1
The party does things that are beneficial for p... I did not vote                                            1
                                                   I wanted the policies I cared about to be actioned        3
                                                   I wanted to support the party I felt has done t...        3
                                                   I wanted to support the party that has helped m...        1
                                                   We needed change in leadership                            1
The party promised to do things in my community... I did not vote                                            1
                                                   I wanted the policies I cared about to be actioned        2
                                                   I wanted to support the party I felt has done t...        2
                                                   I wanted to support the party that has helped m...      

By percentage, how many respondents chose each answer option for the following questions:
    * Compared to six months ago, did any of the following change your mind about whether to vote?
    * Compared to six months ago, did any of the following change your mind about which party to vote for?
    * Compared to six months ago, did any of the following change your mind about reasons for choosing a party to vote for?


In [325]:
data.loc[:, ["question id", "question text"]].drop_duplicates()
Q_1 = 152
Q_2 = 153
Q_3 = 154

q1 = data[data["question id"] == Q_1]
data[data["question id"] == Q_1]\
 .loc[:, ["user_key", "answer"]]\
 .groupby("answer")\
 .agg("count")/len(q1) * 100

data[(data["question id"] == Q_1) | (data["question id"] == Q_2) | (data["question id"] == Q_3)]\
 .loc[:, ["user_key", "question text", "answer"]]\
 .groupby(["question text", "answer"])\
 .agg("count").groupby(level=0).apply(lambda x : x / x.sum()) * 100

user_key
question text                                      answer                                                       
Compared to six months ago, did any of the foll... Something I heard or saw in the media (TV, radi...  25.405405
                                                   Something I heard or saw on social media (twitt...  23.783784
                                                   Something I heard or saw when attending a polit...  15.135135
                                                   Something I read or an activity I did on this a...  15.135135
                                                   Something my friends and family members told me      7.567568
                                                   Talking to a party representative                   12.972973
Compared to six months ago, did any of the foll... Something I heard or saw in the media (TV, radi...  24.468085
                                                   Something I heard or saw on social media (twitt...  22.872340
                                                   Something I heard or saw when attending a polit...  15.957447
                                                   Something I read or an activity I did on this a...  16.489362
                                                   Something my friends and family members told me      8.510638
                                                   Talking to a party representative                   11.702128
Compared to six months ago, did any of the foll... Something I heard or saw in the media (TV, radi...  28.342246
                                                   Something I heard or saw on social media (twitt...  20.855615
                                                   Something I heard or saw when attending a polit...  16.577540
                                                   Something I read or an activity I did on this a...  13.368984
                                                   Something my friends and family members told me      8.021390
                                                   Talking to a party representative                   12.834225

We also need to see if there’s any relation between the planned party to vote for and the actual party voted for by respondents both at national and provincial level by %. For instance, the % of people who had planned to vote for a particular party and actually voted for a different party. On the following questions

* Which party did you originally plan to vote for at national level?
* Which party did you actually vote for at national level?


In [348]:
QN_PLAN = 157
QN_ACTUAL = 158


qn_plan = data[data["question id"] == QN_PLAN].loc[:, ["user_key", "answer_cleaned"]].set_index("user_key")
qn_actual = data[data["question id"] == QN_ACTUAL].loc[:, ["user_key", "answer_cleaned"]].set_index("user_key")

national = qn_plan\
 .join(qn_actual, rsuffix="_after")\
 .rename(columns={"answer_cleaned" : "Planned", "answer_cleaned_after": "Actual"})\
 .reset_index()\
 .groupby(["Planned", "Actual"])\
 .agg("count")\
 .rename(columns={"user_key": "Counts"})

national


Counts
Planned             Actual                     
ACDP                ACDP                      3
ANC                 ANC                      66
                    DA                        3
                    EFF                       9
                    I rather not say it       1
                    None                      8
ATM                 ATM                       1
DA                  ANC                       4
                    DA                        9
                    Good                      1
                    I rather not say it       1
                    IFP                       1
                    None                      1
                    UDM                       1
EFF                 ANC                       4
                    DA                        1
                    EFF                      40
                    None                      5
Good                Good                      1
I rather not say it ANC                       1
                    I rather not say it       4
I wasn't sure       DA                        1
                    EFF                       1
                    None                      1
None                None                      8
PAC                 PAC                       1

* Which party did you originally plan to vote for at provincial level?
* Which party did you actually vote for at provincial level?

In [347]:
QP_PLAN = 159
QP_ACTUAL = 160

qp_plan = data[data["question id"] == QP_PLAN].loc[:, ["user_key", "answer_cleaned"]].set_index("user_key")
qp_actual = data[data["question id"] == QP_ACTUAL].loc[:, ["user_key", "answer_cleaned"]].set_index("user_key")

provincial = qn_plan\
 .join(qn_actual, rsuffix="_after")\
 .rename(columns={"answer_cleaned" : "Planned", "answer_cleaned_after": "Actual"})\
 .reset_index()\
 .groupby(["Planned", "Actual"])\
 .agg("count")\
 .rename(columns={"user_key": "Counts"})

provincial

Counts
Planned             Actual                     
ACDP                ACDP                      3
ANC                 ANC                      66
                    DA                        3
                    EFF                       9
                    I rather not say it       1
                    None                      8
ATM                 ATM                       1
DA                  ANC                       4
                    DA                        9
                    Good                      1
                    I rather not say it       1
                    IFP                       1
                    None                      1
                    UDM                       1
EFF                 ANC                       4
                    DA                        1
                    EFF                      40
                    None                      5
Good                Good                      1
I rather not say it ANC                       1
                    I rather not say it       4
I wasn't sure       DA                        1
                    EFF                       1
                    None                      1
None                None                      8
PAC                 PAC                       1

Would you say that the country is going in the wrong direction or going in the right direction?

In [383]:
qanswers(166)

user_key
question text                                      answer_cleaned                                               
Would you say that the country is going in the ... Going in the right direction                        33.142857
                                                   Going in the wrong direction                         9.714286
                                                   Some things are going right, and some things ar...  57.142857

In [384]:
qanswers(169)

user_key
question text                                      answer_cleaned             
When you think of the institutional and leaders... Getting stronger  45.882353
                                                   Getting weaker    21.764706
                                                   Staying the same  32.352941

In [386]:
qanswers(170)

user_key
question text                                      answer_cleaned             
When you think of social inequality in South Af... Decreasing        23.668639
                                                   Increasing        43.195266
                                                   Staying the same  33.136095

In [387]:
qanswers(171)

user_key
question text                                      answer_cleaned                                               
When you think of how citizens are feeling and ... None of the above                                   11.834320
                                                   Reconciliation (coming together towards a share...  25.443787
                                                   Resentment (complaining and withdrawing into pr...  31.952663
                                                   Resistance (organising and fighting against a s...  30.769231